In [1]:
from numpy.random import seed
seed(1)
import tensorflow 
tensorflow.random.set_seed(1) 
import pandas as pd


2022-07-11 16:00:02.825368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-11 16:00:02.825433: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
pip install scikeras

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install libcudart

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement libcudart (from versions: none)
ERROR: No matching distribution found for libcudart
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
[[60]]

[[60]]

In [4]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /home/alice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
from imblearn.over_sampling import SMOTE
from collections import Counter

In [8]:
from tensorflow import keras

In [9]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D, Convolution1D
from keras.optimizers import Adadelta

In [10]:
from sklearn import metrics

In [11]:
def ageGroups(text):
    if text == '10 à 18 anos' or text=='19 à 25 anos':
        return 0
    elif text=='26 à 30 anos' or text=='30 à 40 anos':
        return 1
    else:
        return 2

In [12]:
def remove_stopwords(text):
    all_words = text.split(" ")
    clean_text = [i for i in all_words if i not in stopwords and i!=""]
    return " ".join(clean_text)

In [13]:
def remove_html_tags(text):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [14]:
def lower_texts(text):
    return text.lower()

In [15]:
def clean_text(text):
    clean_text = remove_stopwords(text)
    clean_text = remove_html_tags(clean_text)
    clean_text = lower_texts(clean_text)
    return clean_text

In [16]:
def genderClassifier(text):
    if text == 'Masculino':
        return 0
    else:
        return 1

In [17]:
from scikeras.wrappers import KerasClassifier

In [18]:
from sklearn.model_selection import GridSearchCV

In [37]:
def create_model(filters = [100], kernel_size = [50], strides = [100], 
                 dropout_rate = 0.5, pool_size = [5], dense_units = 100, max_len = 1000):

    model = Sequential()

    # conv 1
    model.add(Conv1D(filters = filters[0], 
                     kernel_size = kernel_size[0],
                     strides = strides[0], 
                     activation = 'relu', 
                     input_shape = (max_len, 1) ))

    # pooling layer 1
    for i in range(len(pool_size)):
        model.add(MaxPooling1D(pool_size = pool_size[i], strides = 1))
        model.add(Activation('relu'))
    
    #model.add(Activation('relu'))
    
    model.add(Flatten())
    
    if dropout_rate is not None:
        model.add(Dropout(dropout_rate))
        
    model.add(Dense(units = dense_units, activation = 'relu'))
    model.add(Dense(units = 2, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', optimizer = Adadelta(
       learning_rate=1, name="Adadelta"
    ), metrics = ['accuracy'])
    return model

In [20]:
def create_model_age(filters = [100], kernel_size = [50], strides = [100], 
                 dropout_rate = 0.5, pool_size = [5], dense_units = 100, max_len = 1000):

    model = Sequential()

    # conv 1
    model.add(Conv1D(filters = filters[0], 
                     kernel_size = kernel_size[0],
                     strides = strides[0], 
                     activation = 'relu', 
                     input_shape = (max_len, 1) ))

    # pooling layer 1
    for i in range(len(pool_size)):
        model.add(MaxPooling1D(pool_size = pool_size[i], strides = 1))
        model.add(Activation('relu'))
    
    #model.add(Activation('relu'))
    
    model.add(Flatten())
    
    if dropout_rate is not None:
        model.add(Dropout(dropout_rate))
        
    model.add(Dense(units = dense_units, activation = 'relu'))
    model.add(Dense(units = 3, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', optimizer = Adadelta(
       learning_rate=1, name="Adadelta"
    ), metrics = ['accuracy'])
    return model

In [23]:
df_train=pd.read_csv("/home/alice/Pesquisa/tcc-v2/particoes/blogsetbr/houdout/train.csv") #random state is a seed value
df_test=pd.read_csv("/home/alice/Pesquisa/tcc-v2/particoes/blogsetbr/houdout/test.csv")

In [28]:
#gridsearch para genero
X_train = df_train["Texts"].apply(clean_text).to_numpy()
X_test = df_test["Texts"].apply(clean_text).to_numpy()
y_train = df_train["GenderClass"].to_numpy()
y_test = df_test["GenderClass"].to_numpy()

num_words = []
for text in (X_train.tolist()+X_test.tolist()):
    num_words.append(len(text.split()))
mean = sum(num_words)//len(num_words)

train_texts = X_train.tolist()
test_texts = X_test.tolist()

tfidfvec = TfidfVectorizer(max_features = mean, max_df=0.9)
tfidfvec.fit(train_texts)
tfidf_train = tfidfvec.transform(train_texts).toarray()
tfidf_test = tfidfvec.transform(test_texts).toarray()

X_train = tfidf_train.reshape(tfidf_train.shape[0],tfidf_train.shape[1],1)
X_test = tfidf_test.reshape(tfidf_test.shape[0],tfidf_test.shape[1],1)
y_train = keras.utils.to_categorical(y_train,num_classes=2)
y_test = keras.utils.to_categorical(y_test,num_classes=2)

size = X_test.shape[1]

CNN = KerasClassifier(model=create_model, 
                epochs=1,
                dropout_rate=0.1,
                filters = 60,
                kernel_size = 1,
                strides = 2,
                pool_size = 4,
                max_len=X_train.shape[1],
                batch_size=32,
                verbose=0,
                validation_split=0.1
                )

params_grid = dict(
        filters = [[60], [100]],
        kernel_size = [[1], [3], [5]],
        strides = [[1], [2],[4]],
        dropout_rate = [0.1,0.4,0.6],
        pool_size = [[4,2],[4],[8,2],[8]],
        epochs = [1, 10],
        batch_size = [32, 64]
)              

grid_search = GridSearchCV(CNN, 
                           params_grid, 
                           scoring='f1_macro', cv=10, 
                           return_train_score=True
                           )

grid_results = grid_search.fit(X_train, y_train)

print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(params_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

print(grid_search.best_score_)

Best parameters set:
	batch_size: 64
	dropout_rate: 0.1
	epochs: 10
	filters: [60]
	kernel_size: [1]
	pool_size: [4]
	strides: [1]
0.728893797260978


In [29]:
#gridsearch para idade
X_train = df_train["Texts"].apply(clean_text).to_numpy()
X_test = df_test["Texts"].apply(clean_text).to_numpy()
y_train = df_train["AgeClass"].to_numpy()
y_test = df_test["AgeClass"].to_numpy()

num_words = []
for text in (X_train.tolist()+X_test.tolist()):
    num_words.append(len(text.split()))
mean = sum(num_words)//len(num_words)

train_texts = X_train.tolist()
test_texts = X_test.tolist()

tfidfvec = TfidfVectorizer(max_features = mean, max_df=0.9)
tfidfvec.fit(train_texts)
tfidf_train = tfidfvec.transform(train_texts).toarray()
tfidf_test = tfidfvec.transform(test_texts).toarray()

X_train = tfidf_train.reshape(tfidf_train.shape[0],tfidf_train.shape[1],1)
X_test = tfidf_test.reshape(tfidf_test.shape[0],tfidf_test.shape[1],1)
y_train = keras.utils.to_categorical(y_train,num_classes=3)
y_test = keras.utils.to_categorical(y_test,num_classes=3)

size = X_test.shape[1]

CNN = KerasClassifier(model=create_model_age, 
                epochs=1,
                dropout_rate=0.1,
                filters = 60,
                kernel_size = 1,
                strides = 2,
                pool_size = 4,
                max_len=X_train.shape[1],
                batch_size=32,
                verbose=0,
                validation_split=0.1
                )

params_grid = dict(
        filters = [[60], [100]],
        kernel_size = [[1], [3], [5]],
        strides = [[1], [2],[4]],
        dropout_rate = [0.1,0.4,0.6],
        pool_size = [[4,2],[4],[8,2],[8]],
        epochs = [1, 10],
        batch_size = [32, 64]
)              

grid_search = GridSearchCV(CNN, 
                           params_grid, 
                           scoring='f1_macro', cv=10, 
                           return_train_score=True
                           )

grid_results = grid_search.fit(X_train, y_train)

print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(params_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

print(grid_search.best_score_)

Best parameters set:
	batch_size: 32
	dropout_rate: 0.6
	epochs: 10
	filters: [60]
	kernel_size: [5]
	pool_size: [8, 2]
	strides: [4]
0.4535416477980448


In [41]:
import numpy as np

In [44]:
df_train=pd.read_csv("/home/alice/Pesquisa/tcc-v2/particoes/blogsetbr/houdout/train.csv") #random state is a seed value
df_test=pd.read_csv("/home/alice/Pesquisa/tcc-v2/particoes/blogsetbr/houdout/test.csv")

X_train = df_train["Texts"].apply(clean_text).to_numpy()
X_test = df_test["Texts"].apply(clean_text).to_numpy()
y_train_gender = df_train["GenderClass"].to_numpy()
y_test_gender = df_test["GenderClass"].to_numpy()

num_words = []
for text in (X_train.tolist()+X_test.tolist()):
    num_words.append(len(text.split()))

mean = sum(num_words)//len(num_words)

train_texts = X_train.tolist()
test_texts = X_test.tolist()

tfidfvec = TfidfVectorizer(max_features = mean, max_df=0.9)
tfidfvec.fit(train_texts)
tfidf_train = tfidfvec.transform(train_texts).toarray()
tfidf_test = tfidfvec.transform(test_texts).toarray()

X_train = tfidf_train.reshape(tfidf_train.shape[0],tfidf_train.shape[1],1)
X_test = tfidf_test.reshape(tfidf_test.shape[0],tfidf_test.shape[1],1)

y_train_gander_cat = keras.utils.to_categorical(y_train_gender,num_classes=2)
y_test_gender_cat = keras.utils.to_categorical(y_test_gender,num_classes=2)

size = X_test.shape[1]

model_gender = create_model(filters = [100], kernel_size = [3], strides = [1], dropout_rate = 0.4, pool_size = [4], dense_units = 512, max_len = size)

model_gender.fit(X_train,y_train_gander_cat,validation_data=(X_test,y_test_gender_cat), batch_size=32, epochs=2)

y_pred = model_gender.predict(
    X_test
)

y_pred_list = [np.argmax(x, axis=-1) for x in y_pred]

df_test["PredictGender"] = y_pred_list

metrics.f1_score(y_test_gender, y_pred_list, average='macro')

Epoch 1/2
64/64 [==============================] - 47s 724ms/step - loss: 0.6241 - accuracy: 0.6734 - val_loss: 0.8199 - val_accuracy: 0.5049
Epoch 2/2
16/16 [==============================] - 1s 87ms/step


0.7335697409309273

In [46]:
#gridsearch para feminino
df_train_fem = df_train[df_train["GenderClass"]==1]
df_test_fem = df_test[df_test["PredictGender"]==1]

X_train_fem = df_train_fem["Texts"].apply(clean_text).to_numpy()
y_train_fem = df_train_fem["AgeClass"].to_numpy()

X_test_fem = df_test_fem["Texts"].apply(clean_text).to_numpy()
y_test_fem = df_test_fem["AgeClass"].to_numpy()

num_words_fem = []
for text in (X_train_fem.tolist()+X_test_fem.tolist()):
    num_words_fem.append(len(text.split()))
mean_fem = sum(num_words_fem)//len(num_words_fem)

train_texts_fem = X_train_fem.tolist()
test_texts_fem = X_test_fem.tolist()

tfidfvec_fem = TfidfVectorizer(max_features = mean_fem, max_df=0.9)
tfidfvec_fem.fit(train_texts_fem)
tfidf_train_fem = tfidfvec_fem.transform(train_texts_fem).toarray()
tfidf_test_fem = tfidfvec_fem.transform(test_texts_fem).toarray()

X_train_fem = tfidf_train_fem.reshape(tfidf_train_fem.shape[0],tfidf_train_fem.shape[1],1)
X_test_fem = tfidf_test_fem.reshape(tfidf_test_fem.shape[0],tfidf_test_fem.shape[1],1)
y_train_fem = keras.utils.to_categorical(y_train_fem,num_classes=3)
y_test_fem_cat = keras.utils.to_categorical(y_test_fem,num_classes=3)
size_fem = X_test_fem.shape[1]

CNN = KerasClassifier(model=create_model_age, 
                epochs=1,
                dropout_rate=0.1,
                filters = 60,
                kernel_size = 1,
                strides = 2,
                pool_size = 4,
                max_len=X_train_fem.shape[1],
                batch_size=32,
                verbose=0,
                validation_split=0.1
                )

params_grid = dict(
        filters = [[60], [100]],
        kernel_size = [[1], [3], [5]],
        strides = [[1], [2],[4]],
        dropout_rate = [0.1,0.4,0.6],
        pool_size = [[4,2],[4],[8,2],[8]],
        epochs = [1, 10],
        batch_size = [32, 64]
)              

grid_search = GridSearchCV(CNN, 
                           params_grid, 
                           scoring='f1_macro', cv=10, 
                           return_train_score=True
                           )

grid_results = grid_search.fit(X_train_fem, y_train_fem)

print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(params_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

print(grid_search.best_score_)

Best parameters set:
	batch_size: 32
	dropout_rate: 0.6
	epochs: 10
	filters: [60]
	kernel_size: [1]
	pool_size: [4, 2]
	strides: [2]
0.47766647032443615


In [47]:
#gridsearch para masculino
df_train_masc = df_train[df_train["GenderClass"]==0]
df_test_masc = df_test[df_test["PredictGender"]==0]

X_train_masc = df_train_masc["Texts"].apply(clean_text).to_numpy()
y_train_masc = df_train_masc["AgeClass"].to_numpy()

X_test_masc = df_test_masc["Texts"].apply(clean_text).to_numpy()
y_test_masc = df_test_masc["AgeClass"].to_numpy()

num_words_masc = []
for text in (X_train_masc.tolist()+X_test_masc.tolist()):
    num_words_masc.append(len(text.split()))
mean_masc = sum(num_words_masc)//len(num_words_masc)

train_texts_masc = X_train_masc.tolist()
test_texts_masc = X_test_masc.tolist()

tfidfvec_masc = TfidfVectorizer(max_features = mean_masc, max_df=0.9)
tfidfvec_masc.fit(train_texts_masc)
tfidf_train_masc = tfidfvec_masc.transform(train_texts_masc).toarray()
tfidf_test_masc = tfidfvec_masc.transform(test_texts_masc).toarray()

X_train_masc = tfidf_train_masc.reshape(tfidf_train_masc.shape[0],tfidf_train_masc.shape[1],1)
X_test_masc = tfidf_test_masc.reshape(tfidf_test_masc.shape[0],tfidf_test_masc.shape[1],1)
y_train_masc = keras.utils.to_categorical(y_train_masc,num_classes=3)
y_test_masc_cat = keras.utils.to_categorical(y_test_masc,num_classes=3)
size_masc = X_test_masc.shape[1]

CNN = KerasClassifier(model=create_model_age, 
                epochs=1,
                dropout_rate=0.1,
                filters = 60,
                kernel_size = 1,
                strides = 2,
                pool_size = 4,
                max_len=X_train_masc.shape[1],
                batch_size=32,
                verbose=0,
                validation_split=0.1
                )

params_grid = dict(
        filters = [[60], [100]],
        kernel_size = [[1], [3], [5]],
        strides = [[1], [2],[4]],
        dropout_rate = [0.1,0.4,0.6],
        pool_size = [[4,2],[4],[8,2],[8]],
        epochs = [1, 10],
        batch_size = [32, 64]
)              

grid_search = GridSearchCV(CNN, 
                           params_grid, 
                           scoring='f1_macro', cv=10, 
                           return_train_score=True
                           )

grid_results = grid_search.fit(X_train_masc, y_train_masc)

print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(params_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

print(grid_search.best_score_)

Best parameters set:
	batch_size: 32
	dropout_rate: 0.6
	epochs: 10
	filters: [60]
	kernel_size: [1]
	pool_size: [4]
	strides: [1]
0.44442063296605705
